# Basics [Numba]
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [basics_Numba.ipynb](https://github.com/diegoinacio/computer-vision-notebooks/blob/master/Tips-and-Tricks/basics_Numba.ipynb)
---
Basic functions and operations using [Numba](http://numba.pydata.org/), *NumPy* and *Python*.

In [ ]:
import numpy as np

## Installation
---

[Installation](http://numba.pydata.org/numba-doc/latest/user/installing.html) command for *anaconda* and *pip*:

```
$ conda install numba
```

or

```
$ pip install numba
```

In [ ]:
import numba

## Jit
---
Jit is the principal and most fundamental Numba's feature. It is a compiler which basically converts a function into efficient machine code.

In [ ]:
from numba import jit

For example, given a matrix $M$ which is expected the sum of all elements. In a "Pythonic" way, the probom could be solved by:

In [ ]:
def sum_matrix(M):
    N1, N2 = M.shape
    result = 0
    for i in range(N1):
        for j in range(N2):
            result += M[i, j]
    return result

Despite Numpy having a native proper function to get this value, *jit* can converts the function in such as faster as **np.sum**. The procedure is basically to pass the original functions as an argument.

In [ ]:
sum_matrix_jit = jit()(sum_matrix)

In [ ]:
np.random.seed(1234)
M = np.random.random([int(4e3)]*2)
# Time measurement over the situations
print('{:<14} |'.format('sum_matrix'), end=' ')
%timeit -n 1 -r 3 sum_matrix(M)
print('{:<14} |'.format('np.sum'), end=' ')
%timeit -n 5 -r 10 np.sum(M)
print('{:<14} |'.format('sum_matrix_jit'), end=' ')
%timeit -n 5 -r 10 sum_matrix_jit(M)

### Using decorator @jit
---
A common way to work with numba's features is through decorators. The equivalent process to get the function conversion *jit()(sum_matrix)* using decorator would be by:

In [ ]:
@jit
def sum_matrix_jit(M):
    N1, N2 = M.shape
    result = 0
    for i in range(N1):
        for j in range(N2):
            result += M[i, j]
    return result

### nopython and @njit
---
Run *jit* entirely without the involvement of the Python interpreter. This mode promotes a best performance for loop based functions. For any non numerical situation which python interpreter is important, this mode must be avoided.

In [ ]:
from numba import njit

Both decorators **@njit** and **@jit(nopython=True)** are the same.

In [ ]:
@njit # or @jit(nopython=True)
def sum_matrix_njit(M):
    N1, N2 = M.shape
    result = 0
    for i in range(N1):
        for j in range(N2):
            result += M[i, j]
    return result

In [ ]:
# Time measurement
print('{:<15} |'.format('sum_matrix_jit'), end=' ')
%timeit -n 5 -r 10 sum_matrix_jit(M)
print('{:<15} |'.format('sum_matrix_njit'), end=' ')
%timeit -n 5 -r 10 sum_matrix_njit(M)

### *parallel* flag and *prange*
---
Jit has a flag which enables the automatic parallelization, and in addition, Numba has implemented the ability to run loops in parallel using *prange*.

In [ ]:
from numba import prange

In [ ]:
@jit(nopython=True, parallel=True)
def sum_matrix_pjit(M):
    N1, N2 = M.shape
    result = 0
    for i in prange(N1):
        for j in prange(N2):
            result += M[i, j]
    return result

In [ ]:
# Time measurement over the situations
print('{:<15} |'.format('sum_matrix_njit'), end=' ')
%timeit -n 5 -r 10 sum_matrix_njit(M)
print('{:<15} |'.format('sum_matrix_pjit'), end=' ')
%timeit -n 5 -r 10 sum_matrix_pjit(M)

## Vectorization
---
Functions which operate over array elements.

In [ ]:
from numba import void, int64, float64

### @vectorize
---
Efficient way to write functions [ufunc](https://docs.scipy.org/doc/numpy/reference/ufuncs.html) which operate over each element of n-dimensional arrays. The auxiliar arguments can be scalar or other arrays which must have the same dims so each element will obey the same order.

In [ ]:
from numba import vectorize

The following example calculates the [Greatest Common Factor](https://en.wikipedia.org/wiki/Greatest_common_divisor) between each element pair from the matrices $M_1$ and $M_2$. The "*vectorized*" function operates over the relative elements $x_{1,i}$ and $x_{2,i}$.

In [ ]:
@vectorize(
    [int64(int64, int64)],
    target='parallel'
)
def GCF(x1, x2):
    result = 1
    x = min(x1, x2)
    for i in prange(2, x + 1):
        div1 = (x1 % i) == 0
        div2 = (x2 % i) == 0
        if div1 and div2:
            result = i
    return result

In [ ]:
N = 1024
np.random.seed(1234)
M1 = np.random.randint(1, 99, (N, N))
M2 = np.random.randint(1, 99, (N, N))
# Time measurement
%timeit -n 5 -r 10 GCF(M1, M2)

In [ ]:
# Example using 4x30 matrices
np.random.seed(1234)
M1 = np.random.randint(1, 99, (4, 30))
M2 = np.random.randint(1, 99, (4, 30))
# Visualize tables
np.set_printoptions(linewidth=125)
print(M1, end=' M1\n\n')
print(M2, end=' M2\n\n')
print(GCF(M1, M2), end=' GCF')

### @guvectorize
---
Similar to *@vectorize* but operates over arbitrary number of elements of input arrays. One difference is the function doesn't necessarily return a value. Instead, it can take the result as an argument. At its declaration, the function's layout is defined in symbolic form like *'(m,n),(),()->(m)'*, where the first argument '*(m,n)*' is and array with dimensions **m**x**n**, the following two arguments '*()*' are scalar or one-element array and the last argument '*(m)*' is the output, which is an array with size **m**.

In [ ]:
from numba import guvectorize

The following example calculates the *density* of a point based on the tridimensional distances from the other points. The input **P** is a list of *n* 3D coordinates and the output **R** is an array of densities of the points.

In [ ]:
@guvectorize(
    [void(
        float64[:,:],
        int64,
        float64,
        float64[:]
    )],
    '(n,m),(),()->(n)'
)
def density(P, N, r, R):
    for i in prange(N):
        x, y, z = P[i]
        for j in prange(i + 1, N):
            if i == j:
                continue
            u, v, w = P[j]
            dx = abs(x - u)
            if dx > r:
                continue
            dy = abs(y - v)
            if dy > r:
                continue
            dz = abs(z - w)
            if dz > r:
                continue
            d = (dx**2 + dy**2 + dz**2)**0.5
            R[i] += 1 - d/(3*r*r)**0.5
            R[j] += 1 - d/(3*r*r)**0.5

In [ ]:
N = 2500
np.random.seed(1234)
# Gaussian mixture
P = np.vstack((
    np.random.normal((-3,0,0), 1.8, (N, 3))*(1, 1, 1/10),
    np.random.normal((0,4,0), 1.4, (N, 3))*(1, 1, 1/10),
    np.random.normal((3,0,0), 1.0, (N, 3))*(1, 1, 1/10)
))
# Output array
R = np.zeros(3*N)
# Time measurement
%time D = density(P, 3*N, 0.75, R)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = Axes3D(fig)
ax.view_init(elev=60, azim=30)
# Visualize points
sct = ax.scatter(*P.T[:3], s=20, c=D)
ax.set_title('number of points: ' + str(3*N), size=20)
plt.colorbar(sct)
plt.show()

## Stencil
---
Efficient way to create stencil kernels. Similarly to *vectorization*, it operates over elements but with the possibility of moving around the element's neighborhood.

In [ ]:
from numba import stencil

Stencil promotes a very convenient way to produce spatial filtering for image processing.

In [ ]:
from imageio import imread
M = imread('../_data/tomography.png')

### Box Filter
---
The [box filter](https://en.wikipedia.org/wiki/Box_blur) results in the average value from the kernel defined by each pixel's neighborhood. The following example shows the box filtering with a kernel 7x7.


In [ ]:
# Box Filter 7x7
@stencil(
    neighborhood=(
        (-3, 3),
        (-3, 3)
    ))
def box_filter(M):
    output = 0
    for i in range(-3, 4):
        for j in range(-3, 4):
            output += M[i, j]
    return output/49

In [ ]:
%time Mb = box_filter(M)

In [ ]:
fig, (axA, axB) = plt.subplots(1, 2, figsize=(20, 10))
# Plot images
axA.imshow(M)
axA.set_title('Input image')
axB.imshow(Mb)
axB.set_title('Box Filter 7x7')

plt.show()

### Parameterized *box filter*
---
Box filter parameterized by radius $r$, which results in a kernel with diameter $2r + 1$.

In [ ]:
def pbox_filter(M, r):
    @stencil(neighborhood=(
        (-r, r),
        (-r, r)
    ))
    def box_filter(M):
        output = 0
        for i in range(-r, r + 1):
            for j in range(-r, r + 1):
                output += M[i, j]
        return output/(2*r + 1)**2
    return box_filter(M)

In [ ]:
# Init parameters
rA = 2
rB = 9
# Run box filters
%time MpA = pbox_filter(M, rA)
%time MpB = pbox_filter(M, rB)

In [ ]:
fig, (axA, axB) = plt.subplots(1, 2, figsize=(20, 10))
# Plot images
axA.imshow(MpA)
axA.set_title('Box Filter {0}x{0}'.format(rA*2 + 1))
axB.imshow(MpB)
axB.set_title('Box Filter {0}x{0}'.format(rB*2 + 1))

plt.show()